# Genetic Algorithm for the Traveling Salesman Problem

In this notebook, we implement a genetic algorithm to find an approximate solution to the classical traveling salesman problem. We then test it on a few graphs with 10-20 nodes. 

In [ ]:
%matplotlib notebook
import numpy as np
import itertools
import random
import networkx as nx
import matplotlib.pyplot as plt

## Example graph G to work on the Traveling Salesman Problem

In [ ]:
random.seed(0)
G=nx.Graph()
weighted_edges=[('A','B', random.randint(1,10)), ('C','B', random.randint(1,10)), ('D','B',random.randint(1,10)),
                ('A','C', random.randint(1,10)), ('D','E', random.randint(1,10)), ('E','F',random.randint(1,10)),
                ('E','G', random.randint(1,10)), ('E','H', random.randint(1,10)), ('F','H',random.randint(1,10)),
                ('C','H', random.randint(1,10)), ('E','B', random.randint(1,10)), ('H','B',random.randint(1,10)), 
                ('D','G', random.randint(1,10)), ('F','G', random.randint(1,10)), ('F','I',random.randint(1,10)),
                ('H','I', random.randint(1,10)), ('C','I', random.randint(1,10)), ('J','C',random.randint(1,10)), 
                ('J','F', random.randint(1,10)), ('G','I', random.randint(1,10)), ('J','I', random.randint(1,10)) ]
G.add_weighted_edges_from(weighted_edges, weight='weight')

In [ ]:
def plot_graph_with_weights(G):
    print('Weighted graph to solve the TSP on')
    print(nx.info(G))
    pos = nx.spring_layout(G, seed=12)
    plt.figure(figsize=(7,4))    
    nx.draw(G, pos, edge_color='black', width=1, linewidths=5, node_size=300, node_color='lightblue', 
            alpha=1, labels={node: node for node in G.nodes()})
    nx.draw_networkx_edge_labels(G, pos,edge_labels={e: G[e[0]][e[1]]['weight'] for e in G.edges()}, font_color='red')

In [ ]:
plot_graph_with_weights(G)    

## The Traveling Salesman Problem (TSP)

A path that visits every vertex of a graph exactly once is called a __Hamiltonian path__. If it returns at the first vertex, is is a __Hamiltonian cycle__.

__The Problem:__ Given a weighted graph and a source vertex v, find the least expensive Hamiltonian cycle.

A solution to the TSP for the example graph G and source node 'A' is a Hamiltonian cycle ['A','B',...,'A'] so that the accummulated cost, over the edges it travels, is the mimimum possible.

## Implementing a Genetic Algorithm

The first problem we encounter is that it is already difficult to find a Hamiltonian cycle, yet alone the optimal one. For this reason, we first introduce all the missing edges by making the graph complete and giving them a weight of $+\infty$. This way any permutation of the nodes after the source node will produce a Hamiltonian path. (Many of them will now have an infinite cost but this is not a problem because we are already trying to minimize the cost.) 

In [ ]:
def complete_graph_with_infinite_cost_edges(G):
    """Given a weighted graph G, make it complete by giving infinite weight to all the missing edges."""
    G.add_weighted_edges_from([(u, v, np.inf) for (u,v) in nx.non_edges(G)], weight='weight')
    return G

In [ ]:
G=complete_graph_with_infinite_cost_edges(G)

In [ ]:
def plot_complete_graph_with_a_few_weights(G):
    print('Complete graph with a few weights depicted')
    print(nx.info(G))
    plt.figure(figsize=(4,4))   
    pos = nx.circular_layout(G)
    nx.draw(G, pos, edge_color='black', width=1, linewidths=5, node_size=300, node_color='lightblue', 
            alpha=1, labels={node: node for node in G.nodes()})
    nx.draw_networkx_edge_labels(G, pos, edge_labels={e: G[e[0]][e[1]]['weight'] for e in list(G.edges())[:10]}, font_color='red')

In [ ]:
plot_complete_graph_with_a_few_weights(G)

## Fitness function

The main ingredient of a genetic algorithm is the fitness function. This is an optimization problem, so the fitness function here coincides with what we want to optimize, namely the cost of a path from the node 'A' to itself. 

In [ ]:
def cost_of_path(G, path):
    """Given a path ['A','B',...,'A'] of a weighted graph, compute its cost."""
    return sum([G[u][v]['weight'] for (u,v) in zip(path[:-1], path[1:])])

#cost_of_path(G, ['A', 'C', 'J', 'I', 'H', 'F', 'G', 'E', 'D', 'B', 'A'])

## Crossover and Mutation

In [ ]:
def crossover(path1, path2, where=None):
    """"Crossover the two paths. The last part of path2 is kept along with the remaining nodes of path1."""
    if where is None:
        where=int(len(path1)/2)
    new_path_1=path1[:]
    new_path_2=path2[:]
    for node in new_path_2[where:len(path2)-1]:
        new_path_1.remove(node)
    return new_path_1[:-1]+new_path_2[where:]

def mutation(path):
    """"Interchange two random nodes of the path. The source node is excluded."""
    new_path=path[:]
    i=random.randint(1,len(path)-3)
    j=random.randint(i+1,len(path)-2)
    new_path[i], new_path[j]=new_path[j], new_path[i]
    return new_path

def mutated_offspring(path1, path2, where=None):
    return mutation(crossover(path1, path2, where))

## The Genetic Algorithm

The genetic algorithm below has the following parameters:
 - max_gens: The threshold number of generations that will pass before the algorithm stops
 - starting_size: The number of random paths generated at the first generation to start the iterations with. This number is also used at every generation to add some completely random paths in the mix.
 - keep: The number of the best paths kept at the end of every generation to mate in the next generation
 - elite_count: The number of best paths passed directly down to the next generation
 - starting_population: A given list of paths to be added in the first generation

In [ ]:
def genetic_algorithm(G, source_node, max_gens=10, starting_size=1000, keep=50, elite_count=1, starting_population=None):
    """Given a weighted complete graph G, find an approximate solution to the TSP using a genetic process of iteratively 
    producing better and better paths."""
    # Initial population
    to_travel=list(set(G.nodes())-set([source_node]))
    initial_population=[ [source_node]+random.sample(to_travel, len(to_travel))+[source_node] for _ in range(starting_size)]
    # Add a starting population if given
    if starting_population is not None:
        initial_population=initial_population+starting_population
    # Order and keep the best 2*#keep paths
    st_pop_ordered=sorted(initial_population, key=lambda p:cost_of_path(G, p))
    current_population=st_pop_ordered[:2*keep]
    generation=0
    # Generations loop
    while generation<max_gens:
        new_population=[]

        # Creating mutants by crossing over the paths of the last population
        for path1 in current_population:
            for path2 in current_population:
                offspring=mutated_offspring(path1, path2)
                new_population.append(offspring)
                offspring2=mutated_offspring(path1, path2, random.randint(2,len(path1)-2))
                new_population.append(offspring2)

        # Adding the best from the previous generation
        new_population=new_population+current_population[:elite_count]
        # Adding some random solutions
        new_population=new_population+[[source_node]+random.sample(to_travel, len(to_travel))+[source_node] for _ in range(starting_size)]
        # Order and keep the best #keep paths       
        current_population=list(sorted(new_population, key=lambda p:cost_of_path(G, p)))[:keep]
        generation+=1
        # To see progress
        print(f'In generation {generation}, the 10 best fitnesses are:\n', [cost_of_path(G, path) for path in current_population[:10]])
    return cost_of_path(G, current_population[0]), current_population[0]

The algorithm above, as many genetic algorithms and Machine Learning algorithms in general, has a high tendency to get stuck in local mimima. One possible approach around that is the algorithm below. It runs the genetic algorithm many times, keeps the best paths found each time, and then starts the algorithm one last time using those paths in the initial population. It has the following additional parameters:
 - final_max_gens: The threshold number of generations that will pass before the algorithm stops at the last step where the best paths are collected
 - reruns: The number of times the genetic algorithm will start fresh

In [ ]:
def genetic_algorithm_best_from_reruns(G, source_node, max_gens=10, starting_size=1000, keep=50, elite_count=1, final_max_gens=20, reruns=5):
    """Rerun the genetic algorithm many times to find good solutions. Run one last time using them as a starting population."""
    good=[]
    for _ in range(reruns):
        good.append(genetic_algorithm(G, source_node, max_gens, starting_size, keep, elite_count)[1])
    print(f'\nBelow the previous {reruns} best solutions will be used in the initial population\n')    
    return genetic_algorithm(G, source_node, final_max_gens, starting_size, keep, elite_count, good)              

In [ ]:
genetic_algorithm(G, 'A')

In [ ]:
genetic_algorithm_best_from_reruns(G, 'A', max_gens=3, starting_size=1000, keep=50, elite_count=2, final_max_gens=10, reruns=5)

## Brute force solution for the graph G

Here we present a brute force approach to the TSP in order to compare the paths found. For n nodes, there are (n-1)! cycles from 'A' back to 'A'. For n=10, that is 362880 possible paths. For more nodes than that, the brute force approach becomes inefficient. 

In [ ]:
def tsp_by_brute_force(G, source_node):
    n=len(G.nodes())
    if n>12:
        return f'Brute force approach is inefficient for {n} nodes.'
    to_travel=list(set(G.nodes())-set([source_node]))
    minimum_path=[source_node]+random.sample(to_travel, len(to_travel))+[source_node]
    minimum_cost=cost_of_path(G, minimum_path)
    for perm in itertools.permutations(to_travel):
        inside_path=list(perm)
        if cost_of_path(G, [source_node]+inside_path+[source_node])<minimum_cost:
            minimum_path=[source_node]+inside_path+[source_node]
            minimum_cost=cost_of_path(G, [source_node]+inside_path+[source_node])
    return minimum_cost, minimum_path

In [ ]:
tsp_by_brute_force(G, 'A')

For the example graph G, we see that the genetic algorithm finds the best possible solution. In the additional examples below here the number of nodes is much higher, the algorithm will only find good inexpensive paths but with no guarantee that they are the best possible. 

## More examples

##### Example 1
Complete graph with 20 vertices and finite weights.

In [ ]:
random.seed(0)
G1=nx.gnm_random_graph(20, 190, seed=0)
for (u, v) in G1.edges():
    G1.edges[u,v]['weight'] = random.randint(1,100)
#plot_complete_graph_with_a_few_weights(G1)

In [ ]:
genetic_algorithm(G1, 0, max_gens=10, starting_size=1000, keep=50, elite_count=1, starting_population=None)

In [ ]:
genetic_algorithm_best_from_reruns(G1, 0, max_gens=10, starting_size=1000, keep=50, elite_count=1, final_max_gens=20, reruns=5)

##### Example 2
Erdos-Renyi graph with 12 nodes and 60 edges.

In [ ]:
random.seed(0)
G2=nx.gnm_random_graph(12, 60, seed=0)
for (u, v) in G2.edges():
    G2.edges[u,v]['weight'] = random.randint(1,100)
plot_graph_with_weights(G2)

In [ ]:
G2=complete_graph_with_infinite_cost_edges(G2)    
plot_complete_graph_with_a_few_weights(G2)

In [ ]:
genetic_algorithm(G2, 0, max_gens=10, starting_size=10000, keep=50, elite_count=5, starting_population=None)

In [ ]:
genetic_algorithm_best_from_reruns(G2, 0, max_gens=3, starting_size=10000, keep=50, elite_count=1, final_max_gens=10, reruns=10)

In [ ]:
#With 12 nodes the brute force apporach takes a very long time (around 30-60 minutes) but still reasonable.
#tsp_by_brute_force(G2, 0)

##### Example 3
Erdos-Renyi graph with 30 nodes and 300 edges.

In [ ]:
random.seed(0)
G3=nx.gnm_random_graph(30, 300, seed=0)
for (u, v) in G3.edges():
    G3.edges[u,v]['weight'] = random.randint(1,100)
#plot_graph_with_weights(G3)

In [ ]:
G3=complete_graph_with_infinite_cost_edges(G3)    
#plot_complete_graph_with_a_few_weights(G3)

In [ ]:
genetic_algorithm(G3, 0, max_gens=10, starting_size=100, keep=50, elite_count=1, starting_population=None)

In [ ]:
genetic_algorithm_best_from_reruns(G3, 0, max_gens=10, starting_size=10000, keep=50, elite_count=1, final_max_gens=10, reruns=5)

##### End of notebook